# Part 4, Topic 1: Power and Hamming Weight Relationship

---
NOTE: This lab references some (commercial) training material on [ChipWhisperer.io](https://www.ChipWhisperer.io). You can freely execute and use the lab per the open-source license (including using it in your own courses if you distribute similarly), but you must maintain notice about this source location. Consider joining our training course to enjoy the full experience.

---

**SUMMARY:** *In the previous part of SCA101, we used the fact a relationship exists between the value of the bits being manipulated on a microcontroller to recover an AES key*

*In this lab, we'll improve our model by looking at how power relates to hamming weight.*

**LEARNING OUTCOMES:**

* Understand hamming weight
* Calculate hamming weight of a byte
* Identify the point where the SBox output is being written
* Plot power consumption at the SBox output vs. hamming weight


## AES Model

We'll be looking at the SBox again, so grab your implementation from the last section:

In [83]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
CRYPTO_TARGET = 'TINYAES128C'
SS_VER='SS_VER_2_1'

In [84]:
%run "../../Setup_Scripts/Setup_Generic.ipynb"

INFO: Found ChipWhisperer😍


In [85]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER"
cd ../../../firmware/mcu/simpleserial-aes
make PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3 -j

Building for platform CWLITEARM with CRYPTO_TARGET=TINYAES128C
SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
Blank crypto options, building for AES128
.
arm-none-eabi-gcc (15:13.2.rel1-2) 13.2.1 20231009
Copyright (C) 2023 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

Welcome to another exciting ChipWhisperer target build!!
Size after:
+--------------------------------------------------------
+ Built for platform CW-Lite Arm \(STM32F3\) with:
   text	   data	    bss	    dec	    hex	filename
   5784	    532	   1572	   7888	   1ed0	simpleserial-aes-CWLITEARM.elf
+ CRYPTO_TARGET = TINYAES128C
+ CRYPTO_OPTIONS = AES128C
+--------------------------------------------------------


In [86]:
cw.program_target(scope, prog, "../../../firmware/mcu/simpleserial-aes/simpleserial-aes-{}.hex".format(PLATFORM))

Detected known STMF32: STM32F302xB(C)/303xB(C)
Extended erase (0x44), this can take ten seconds or more
Attempting to program 6315 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 6315 bytes


In [87]:
splot = cw.StreamPlot()
splot.plot()

:DynamicMap   []
   :Curve   [x]   (y)

In [142]:
dir(ktp)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_fixedKey',
 '_fixedPlain',
 '_initPattern',
 '_key',
 '_key_len',
 '_name',
 '_text_len',
 '_textin',
 'fixed_key',
 'fixed_text',
 'getInitialKey',
 'getInitialText',
 'getKeyType',
 'getPlainType',
 'get_key_type',
 'init',
 'initPair',
 'init_pair',
 'initkey',
 'inittext',
 'keyLen',
 'key_len',
 'newPair',
 'new_pair',
 'next',
 'next_key',
 'next_text',
 'setInitialKey',
 'setInitialText',
 'setKeyType',
 'setPlainType',
 'setTarget',
 'set_key_type',
 'textLen',
 'text_len',
 'types',
 'validateKey',
 'validateText']

In [151]:
ktp2 = cw.ktp.Basic()
ktp2.fixed_key = False

In [159]:
key, text = ktp2.next()
print(key)
print(text)

CWbytearray(b'8a 80 08 bc 52 47 0b 1d a7 e0 55 e5 e0 4d 84 87')
CWbytearray(b'bf c4 b3 a1 b9 0a 68 3b fe b8 fe 9c dc 0a 39 5c')


In [153]:
key, text = ktp.next()
print(key)
print(text)

CWbytearray(b'2b 7e 15 16 28 ae d2 a6 ab f7 15 88 09 cf 4f 3c')
CWbytearray(b'27 2d 33 5a 34 7f bd 89 c4 3a 0f 23 93 68 4c a6')


In [118]:
from tqdm.notebook import trange
import numpy as np
import time

ktp = cw.ktp.Basic()
trace_array = []
textin_array = []

key, text = ktp.next()

target.set_key(key)

N = 50 #increase nano reliability
# if PLATFORM=="CWNANO":
#     N = 200
for i in trange(N, desc='Capturing traces'):
    scope.arm()
    
    target.simpleserial_write('p', text)
    
    ret = scope.capture()
    if ret:
        print("Target timed out!")
        continue
    
    response = target.simpleserial_read('r', 16)
    
    trace_array.append(scope.get_last_trace())
    textin_array.append(text)
    
    key, text = ktp.next()
    splot.update(scope.get_last_trace())
    
trace_array = np.array(trace_array)

Capturing traces:   0%|          | 0/50 [00:00<?, ?it/s]

OSError: Scope is not connected. Connect it first...

In [160]:
import pickle

with open('data.pkl', 'wb') as f:
    pickle.dump({'trace_array': trace_array, 'textin_array': textin_array, 'textout_array': textout_array, 'key_array': key_array}, f)

In [211]:
from tqdm.notebook import trange
import numpy as np
import time
import pickle

In [212]:
# with open('traces_for_assignmnets/data1.pkl', 'rb') as f:
with open('traces_for_assignmnets/data7.pkl', 'rb') as f:
    data = pickle.load(f)

trace_array = data['trace_array']
textin_array = data['textin_array']
textout_array = data['textout_array']
key_array = data['key_array']

In [213]:
trace_array

array([[ 0.09570312, -0.05859375, -0.03125   , ...,  0.00488281,
         0.01269531,  0.01074219],
       [ 0.10253906, -0.05664062, -0.02734375, ..., -0.00195312,
         0.0078125 ,  0.00585938],
       [ 0.09960938, -0.05371094, -0.03125   , ..., -0.00195312,
         0.00683594,  0.00683594],
       ...,
       [ 0.09667969, -0.05566406, -0.03125   , ...,  0.0078125 ,
         0.01464844,  0.01269531],
       [ 0.09863281, -0.05371094, -0.03125   , ...,  0.00683594,
         0.01269531,  0.01367188],
       [ 0.09960938, -0.05664062, -0.03027344, ...,  0.00488281,
         0.01074219,  0.01171875]])

In [214]:
textin_array

[CWbytearray(b'1d 8e b5 1c 75 7c 9d 4a d0 fe 80 27 8d 1a 09 16'),
 CWbytearray(b'65 3a c1 d1 16 84 6c dc 38 20 ab ba 7c 4d 2c 8b'),
 CWbytearray(b'60 67 8a 07 3e 12 a3 83 c6 be a3 8c a0 b1 24 66'),
 CWbytearray(b'80 45 a8 a5 99 12 9f 88 3b 4c d8 6a 4b bf 9d 4a'),
 CWbytearray(b'19 6e e3 0d e8 33 d3 ae e9 d1 77 b3 6e 43 b3 ef'),
 CWbytearray(b'e7 ae 56 1b 4f 2e fc c4 05 78 0d 3b e7 af 1c 64'),
 CWbytearray(b'fd 49 cc 2c 1e bb 3a b8 69 4d fa b2 c2 45 dc 7f'),
 CWbytearray(b'49 1d be 77 40 ac d7 73 bb 1a 5c 3b cb bb b2 53'),
 CWbytearray(b'8f 9e ab 90 6e df d2 00 a7 13 19 a2 dc 5f 7b 8e'),
 CWbytearray(b'd7 4a e9 8d 35 df ba a8 57 da 7a c8 42 49 71 d7'),
 CWbytearray(b'09 56 3a 16 9c 77 9a ee 70 4a 40 36 e0 ec 40 04'),
 CWbytearray(b'95 ca ef bc c6 f7 5d 35 2a 68 e8 dc b9 49 e8 b2'),
 CWbytearray(b'a4 c9 28 65 09 6f 81 48 f9 32 7c cb 02 f3 a0 85'),
 CWbytearray(b'76 d9 06 59 90 58 42 57 b9 ad 58 69 b2 ff c4 2a'),
 CWbytearray(b'24 c3 9e 16 f1 57 2f d6 ba 42 7c b0 46 4f e3 54'),
 CWbytearr

In [215]:
textout_array

[CWbytearray(b'71 70 a6 6b e2 e8 6e ba 6c 87 9b eb e9 5e ba af'),
 CWbytearray(b'5c 6f 4d 0a 2d c5 e2 46 ba 02 86 92 b0 fc 9b 41'),
 CWbytearray(b'ae da 9c 97 a2 de 55 27 ec 57 68 61 f1 75 d7 06'),
 CWbytearray(b'1d f3 5e 8a a2 ec a5 e3 f6 b1 2d cf 2b 38 95 08'),
 CWbytearray(b'4b a4 06 70 de 66 47 17 63 b0 49 3c 8d 34 5e b9'),
 CWbytearray(b'59 66 8f 11 d4 ac c3 2c a5 74 e2 e5 dc 1b 62 09'),
 CWbytearray(b'67 84 bd e1 e1 0b 8c d3 e6 7b d9 03 5d 3f 08 b4'),
 CWbytearray(b'59 9a 81 e3 fd 7c 52 33 72 67 e5 60 a9 c8 62 db'),
 CWbytearray(b'db 66 ee 2c 9f e5 a0 5e dc f1 07 c1 a6 bb b2 39'),
 CWbytearray(b'd0 fb 00 41 b4 ec dd 30 02 1e ee 03 26 e0 18 fa'),
 CWbytearray(b'73 27 7f 52 2d 87 89 51 6e 1d 71 81 cc 5e 2c b7'),
 CWbytearray(b'76 f6 9e e3 f8 0b db b7 98 4e 63 3a 73 8e 8c a5'),
 CWbytearray(b'8e c1 2b 86 6f c5 c5 ba 4f ec 08 d8 40 54 54 a5'),
 CWbytearray(b'20 c8 1c 09 09 a7 64 88 e1 48 d3 36 44 17 1c 27'),
 CWbytearray(b'3b 0e 8c 9d 05 42 ed 58 81 e4 7c 7d 7d 55 08 c6'),
 CWbytearr

In [216]:
key_array

[CWbytearray(b'b9 99 b8 6d 79 f9 46 e9 f9 e6 89 d6 ee aa e8 ec'),
 CWbytearray(b'b9 99 b8 6d 79 f9 46 e9 f9 e6 89 d6 ee aa e8 ec'),
 CWbytearray(b'b9 99 b8 6d 79 f9 46 e9 f9 e6 89 d6 ee aa e8 ec'),
 CWbytearray(b'b9 99 b8 6d 79 f9 46 e9 f9 e6 89 d6 ee aa e8 ec'),
 CWbytearray(b'b9 99 b8 6d 79 f9 46 e9 f9 e6 89 d6 ee aa e8 ec'),
 CWbytearray(b'b9 99 b8 6d 79 f9 46 e9 f9 e6 89 d6 ee aa e8 ec'),
 CWbytearray(b'b9 99 b8 6d 79 f9 46 e9 f9 e6 89 d6 ee aa e8 ec'),
 CWbytearray(b'b9 99 b8 6d 79 f9 46 e9 f9 e6 89 d6 ee aa e8 ec'),
 CWbytearray(b'b9 99 b8 6d 79 f9 46 e9 f9 e6 89 d6 ee aa e8 ec'),
 CWbytearray(b'b9 99 b8 6d 79 f9 46 e9 f9 e6 89 d6 ee aa e8 ec'),
 CWbytearray(b'b9 99 b8 6d 79 f9 46 e9 f9 e6 89 d6 ee aa e8 ec'),
 CWbytearray(b'b9 99 b8 6d 79 f9 46 e9 f9 e6 89 d6 ee aa e8 ec'),
 CWbytearray(b'b9 99 b8 6d 79 f9 46 e9 f9 e6 89 d6 ee aa e8 ec'),
 CWbytearray(b'b9 99 b8 6d 79 f9 46 e9 f9 e6 89 d6 ee aa e8 ec'),
 CWbytearray(b'b9 99 b8 6d 79 f9 46 e9 f9 e6 89 d6 ee aa e8 ec'),
 CWbytearr

In [217]:
%matplotlib notebook
import matplotlib.pylab as plt

# ###################
# START SOLUTION
# ###################
plt.figure()
plt.plot(trace_array[0], 'r')
plt.plot(trace_array[1], 'g')
plt.show()
# ###################
# END SOLUTION
# ###################

<IPython.core.display.Javascript object>

In [218]:
numtraces = np.shape(trace_array)[0] #total number of traces
numpoints = np.shape(trace_array)[1] #samples per trace
print(numtraces, numpoints)

50 5000


In [219]:
scope.dis()
target.dis()

(ChipWhisperer Scope ERROR|File naeusbchip.py:108) Scope already disconnected!


In [220]:
# ###################
# Add your code here
sbox = [
    # 0    1    2    3    4    5    6    7    8    9    a    b    c    d    e    f 
    0x63,0x7c,0x77,0x7b,0xf2,0x6b,0x6f,0xc5,0x30,0x01,0x67,0x2b,0xfe,0xd7,0xab,0x76, # 0
    0xca,0x82,0xc9,0x7d,0xfa,0x59,0x47,0xf0,0xad,0xd4,0xa2,0xaf,0x9c,0xa4,0x72,0xc0, # 1
    0xb7,0xfd,0x93,0x26,0x36,0x3f,0xf7,0xcc,0x34,0xa5,0xe5,0xf1,0x71,0xd8,0x31,0x15, # 2
    0x04,0xc7,0x23,0xc3,0x18,0x96,0x05,0x9a,0x07,0x12,0x80,0xe2,0xeb,0x27,0xb2,0x75, # 3
    0x09,0x83,0x2c,0x1a,0x1b,0x6e,0x5a,0xa0,0x52,0x3b,0xd6,0xb3,0x29,0xe3,0x2f,0x84, # 4
    0x53,0xd1,0x00,0xed,0x20,0xfc,0xb1,0x5b,0x6a,0xcb,0xbe,0x39,0x4a,0x4c,0x58,0xcf, # 5
    0xd0,0xef,0xaa,0xfb,0x43,0x4d,0x33,0x85,0x45,0xf9,0x02,0x7f,0x50,0x3c,0x9f,0xa8, # 6
    0x51,0xa3,0x40,0x8f,0x92,0x9d,0x38,0xf5,0xbc,0xb6,0xda,0x21,0x10,0xff,0xf3,0xd2, # 7
    0xcd,0x0c,0x13,0xec,0x5f,0x97,0x44,0x17,0xc4,0xa7,0x7e,0x3d,0x64,0x5d,0x19,0x73, # 8
    0x60,0x81,0x4f,0xdc,0x22,0x2a,0x90,0x88,0x46,0xee,0xb8,0x14,0xde,0x5e,0x0b,0xdb, # 9
    0xe0,0x32,0x3a,0x0a,0x49,0x06,0x24,0x5c,0xc2,0xd3,0xac,0x62,0x91,0x95,0xe4,0x79, # a
    0xe7,0xc8,0x37,0x6d,0x8d,0xd5,0x4e,0xa9,0x6c,0x56,0xf4,0xea,0x65,0x7a,0xae,0x08, # b
    0xba,0x78,0x25,0x2e,0x1c,0xa6,0xb4,0xc6,0xe8,0xdd,0x74,0x1f,0x4b,0xbd,0x8b,0x8a, # c
    0x70,0x3e,0xb5,0x66,0x48,0x03,0xf6,0x0e,0x61,0x35,0x57,0xb9,0x86,0xc1,0x1d,0x9e, # d
    0xe1,0xf8,0x98,0x11,0x69,0xd9,0x8e,0x94,0x9b,0x1e,0x87,0xe9,0xce,0x55,0x28,0xdf, # e
    0x8c,0xa1,0x89,0x0d,0xbf,0xe6,0x42,0x68,0x41,0x99,0x2d,0x0f,0xb0,0x54,0xbb,0x16  # f
]

def aes_internal(inputdata, key):
    return sbox[inputdata ^ key]
# ###################
#raise NotImplementedError("Add your code here, and delete this.")

In [221]:
#Simple test vectors - if you get the check-mark printed all OK.
assert(aes_internal(0xAB, 0xEF) == 0x1B)
assert(aes_internal(0x22, 0x01) == 0x26)
print("✔️ OK to continue!")

✔️ OK to continue!


## Hamming Weight

Recall that the reason that there's a relationship between power consumption and the microcontroller's internal data is that setting this data takes power. We've also seen that the more data that is set, the greater the average power draw. It's not far fetched, then, that there should be some sort of consistant relationship between the number of bits set to 1, called the **Hamming weight** and the power consumed by doing so.

Hamming weight, despite being a pretty simple idea, actually isn't trivial to calculate (see https://en.wikipedia.org/wiki/Hamming_weight#Efficient_implementation). You can write a function to do this, but in Python it's far easier to just convert to a string of bits and count the `"1"`s:

In [222]:
def calc_hamming_weight(n):
    return bin(n).count("1")

Even better, create a lookup table (aka do the calculation for each number between 0 and 255 and stick them in an array):

In [223]:
# ###################
# Add your code here
HW = [bin(n).count("1") for n in range(0, 256)]
# ###################
#raise NotImplementedError("Add Your Code Here")

In [224]:
int('0x53',16)

83

In [225]:
HW[83]

4

In [226]:
assert HW[0x53] == 4
print("✔️ OK to continue!")

✔️ OK to continue!


Our first issue that we run into is that we don't know where the SBox operation is happening. It should be happening pretty close to the beginning (let's guess and say within the first 2000 samples). One thought is that we could group the traces by hamming weight and assign a colour to each one. If we plot that, we might be able to find a pattern:

In [227]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.palettes import brewer

output_notebook()
p = figure()

plot_start = 0
plot_end = 2000
xrange = list(range(len(trace_array[0]))[plot_start:plot_end])
bnum = 0
color_mapper = brewer['PRGn'][9]

for tnum in range(len(trace_array)):
    hw_of_byte = HW[aes_internal(textin_array[tnum][bnum], key[bnum])]
    p.line(xrange, trace_array[tnum][plot_start:plot_end], line_color=color_mapper[hw_of_byte])
# tnum=7
# hw_of_byte = HW[aes_internal(textin_array[tnum][bnum], key[bnum])]
# p.line(xrange, trace_array[tnum][plot_start:plot_end], line_color=color_mapper[hw_of_byte])
# print(hw_of_byte)
show(p)


Loading BokehJS ...

In [228]:
#we thought tha by dividing based on hamming weight , we might be able to get the power correlation . but we didnt get that yet

Unfortunately, you'll probably find that this plot doesn't really tell us much; the part of power consumption associated with the SBox output is just too small to pick out. We could try averaging the hamming weight groups to make things more distinct, but that doesn't solve the fundamental issue of the SBox output being lost in the noise of everything else happening on the chip.

Instead, let's approach this from a different angle. Really, what we want here is to remove the overall "shape" of the trace and just leave the signal from the SBox output. We could just pick a trace and subtract it from each group, but subtracting an average of all the traces instead will make the plot more distinct. Even better would be to have an even weighting between all of the hamming weight groups, since the extreme hamming weights (0 and 8) are far less common than the middle values, but this won't end up being super necessary (though you can still attempt this if you'd like).  The plot will also be more distinct (and plot a lot faster) if we average all the hamming weight groups to remove any outliers as well. Try implementing this (we'll again handle the plotting for you):

In [229]:
hw_of_byte = HW[aes_internal(textin_array[tnum][bnum], key[bnum])]
hw_of_byte
trace_array[tnum]

array([ 0.09960938, -0.05664062, -0.03027344, ...,  0.00488281,
        0.01074219,  0.01171875])

In [230]:
hw_groups[2][0]

array([ 0.09570312, -0.05859375, -0.03125   , ...,  0.00488281,
        0.01269531,  0.01074219])

In [231]:
# ###################
# Add your code here
# ###################
#raise NotImplementedError("Add Your Code Here")

# ###################
# START SOLUTION
# ###################
output_notebook()
p = figure()
hw_groups = [[], [], [], [], [], [], [], [], []]
for tnum in range(len(trace_array)):
    hw_of_byte = HW[aes_internal(textin_array[tnum][bnum], key[bnum])]
    hw_groups[hw_of_byte].append(trace_array[tnum])
    print(textin_array[tnum][bnum],'with', key[bnum],'gives',aes_internal(textin_array[tnum][bnum], key[bnum]),"goes to",hw_of_byte,"group")

# for i in range(9):
#     print(hw_groups[i])
#     print("\n")
hw_groups[0] = np.array([0]* 5000)
hw_groups[8] = np.array([0]* 5000)
print(len(hw_groups[0]))
print(len(hw_groups[1]))
print(len(hw_groups[2]))
print(len(hw_groups[3]))
print(len(hw_groups[4]))
print(len(hw_groups[5]))
print(len(hw_groups[6]))
#print(hw_groups[1][0])
print(len(hw_groups[7]))
print(len(hw_groups[8]))

hw_averages = np.array([np.average(hw_groups[hw], axis=0) for hw in range(1,8)])
# hw =8
# hw_averages[hw] = np.array(np.average(hw_groups[hw], axis=0))
# print(hw_averages[hw])
# print(len(hw_averages[hw]))
avg_trace = np.average(hw_averages, axis=0)
# ###################
# END SOLUTION
# ###################
#print(hw_averages[6])
xrange = list(range(len(trace_array[0]))[plot_start:plot_end])
color_mapper = ['green','red','blue','pink','yellow','orange','magenta']
for hw in range(7):  
    p.line(xrange, (hw_averages[hw]-avg_trace)[plot_start:plot_end], line_color=color_mapper[hw])
#hw = 2
#p.line(xrange, (hw_averages[hw]-avg_trace)[plot_start:plot_end], line_color=color_mapper[hw])
    
show(p)
# color_mapper = ['green','red','blue','pink','yellow','orange','magenta']
# hw = 6
# xrange = list(range(len(trace_array[0]))[plot_start:plot_end])
# p.line(xrange, (hw_averages[hw]-avg_trace)[plot_start:plot_end], line_color='magenta')#color_mapper[hw])
# show(p)

Loading BokehJS ...

29 with 138 gives 136 goes to 2 group
101 with 138 gives 223 goes to 7 group
96 with 138 gives 135 goes to 4 group
128 with 138 gives 103 goes to 5 group
25 with 138 gives 220 goes to 5 group
231 with 138 gives 60 goes to 4 group
253 with 138 gives 245 goes to 6 group
73 with 138 gives 46 goes to 4 group
143 with 138 gives 107 goes to 5 group
215 with 138 gives 76 goes to 3 group
9 with 138 gives 236 goes to 5 group
149 with 138 gives 192 goes to 2 group
164 with 138 gives 49 goes to 3 group
118 with 138 gives 176 goes to 3 group
36 with 138 gives 228 goes to 4 group
0 with 138 gives 126 goes to 6 group
246 with 138 gives 16 goes to 1 group
108 with 138 gives 142 goes to 4 group
139 with 138 gives 124 goes to 5 group
48 with 138 gives 244 goes to 5 group
217 with 138 gives 237 goes to 6 group
174 with 138 gives 54 goes to 4 group
200 with 138 gives 44 goes to 3 group
74 with 138 gives 186 goes to 5 group
9 with 138 gives 236 goes to 5 group
13 with 138 gives 23 goes to 4 group
67 with 

Plotting this, you should get a very distinct spot where the colours separate. This is where the SBox operation is occuring. In fact, it's probably distinct enough that you can choose the SBox loction solely by where the graph is largest:

In [187]:
sbox_loc = np.argmax(abs(hw_averages[0]-avg_trace))
print(sbox_loc)

981


Now that we know where the SBox operation is happening, plot the hamming weight averages by their hamming weight at `sboc_loc`.

**HINT: You may want to convert your hw_averages to a numpy array to allow you to access by column. `hw_averages[:,sbox_loc]` will give you `hw_averages` at the sbox_loc.**

In [188]:
# ###################
# Add your code here
# ###################
# raise NotImplementedError("Add Your Code Here")
output_notebook()
p = figure(title="HW vs Voltage Measurement")
#p.line(range(0, 8), hw_averages[:,sbox_loc], line_color="red")
sbox_locs = [int(np.argmax(abs(hw_averages[hw]-avg_trace))) for hw in range(7)]
print(sbox_locs)
#[plot_start:plot_end]
# for hw in range(7):
x_axis = [1,2,3,4,5,6,7]
p.line(x_axis,sbox_locs, line_color="red")
#p.line(xrange, (hw_averages[hw]-avg_trace)[plot_start:plot_end], line_color=color_mapper[hw])
p.xaxis.axis_label = "Hamming Weight of Intermediate Value"
p.yaxis.axis_label = "Average Value of Measurement"
show(p)
# ###################
# END SOLUTION
# ###################

Loading BokehJS ...

[981, 4625, 413, 929, 153, 3829, 333]


You should find that the relationship is mostly linear, which probably won't come out of left field. It makes sense that setting 8 data bits will take roughly 8x the power that setting one does. 

You will likely also find that the slope of the relationship is negative, unless you're on the ChipWhisperer Nano. This happens for a good reason. If you remember how we are measuring the current into the device, you'll find out that the voltage will go DOWN for an INCREASE in current. You can see this in the following figure:

```     
        Rshunt
(Vin)----v^v^v^----------||------(To ChipWhisperer)
                |
                |
       (To microcontroller)
```

For the ChipWhisperer Nano, the slope is positive due to the presence of an inverting amplifier on the input of the measurement port.

We are measuring the drop across the shunt resistor. An increase in the current causes a higher voltage across the resistor. When no current flows there is no drop across the resistor. But since we only measure a single end of the resistor, we see a higher voltage when no current flows.

We can fix the slope by simply inverting the measurement direction (adding a - in front of the measurement).

Now that we know where the SBox operation is happening, try going back to the original plot and zoom in to that section. Can you pick out the difference between the hamming weights now?

## Conclusions & Next Steps

With this lab, you should be reasonably convinced that there is a linear relationship between the hamming weight of data being set in a microcontroller and the power it consumes from doing so.

In the next lab, we'll see how this can be used to greatly improve over our DPA attack.

---
<small>NO-FUN DISCLAIMER: This material is Copyright (C) NewAE Technology Inc., 2015-2020. ChipWhisperer is a trademark of NewAE Technology Inc., claimed in all jurisdictions, and registered in at least the United States of America, European Union, and Peoples Republic of China.

Tutorials derived from our open-source work must be released under the associated open-source license, and notice of the source must be *clearly displayed*. Only original copyright holders may license or authorize other distribution - while NewAE Technology Inc. holds the copyright for many tutorials, the github repository includes community contributions which we cannot license under special terms and **must** be maintained as an open-source release. Please contact us for special permissions (where possible).

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.</small>